Topics
--------
1. TensorFlow Graphs
2. Adding Inputs with Placeholder nodes
3. Variables
4. Organizing your graph with name scopes


## 1. TensorFlow graphs

In the previous tutorial, we've only referenced *the graph* as some sort of abstract, and we havent questioned how Operations are automatically attached to a graph when we use it. 

Now let's take a look at the [TensorFlow Graph Object](https://www.tensorflow.org/versions/master/api_docs/python/framework/#Graph), learn how to create more of them, use multiple graphs in conjunction with one another.

In [1]:
import tensorflow as tf

In [2]:
# Create a new graph:
g = tf.Graph()

Once we have our Graph initialized, we can add Operations to it by using the ```python Graph.as_default()``` method to access its context manager.

In [3]:
with g.as_default():
    # Create operations as usual; they will be added to graph 'g'
    a = tf.mul(2, 5)

We don't need to specify the graph we'd like to add our Ops. As a convenience, TensorFlow automatically creates a *Graph* when the library is  loaded and assigns it to be the default. Thus, any operations, tensors, etc. define outside of a ```python Graph.as_default()``` context manager will automatically be placed in the default graph:

In [4]:
# Placed in the default graph
in_default_graph = tf.add(2, 3)

# Placed in graph 'g'
with g.as_default():
    in_graph_g = tf.mul(2, 3)
    
# We are no longer in the 'with' block, so this is placed in the default 
# graph
also_in_default_graph = tf.sub(10, 5)


If you'd like to get a handle to the default graph, use the ```python tf.get_default_graph()``` function: ```default_graph = tf.get_default_graph()```

- In most TensorFlow programs, we only ever deal with the default graph. However, creating multiple graphs can be useful if you are defining multiple models that do not have interdependencies. 

- When defining multiple graphs in one file, it's recommended to either not use the default graph or immediately assign to handle to it. This ensures that nodes are added to each group in a uniform manner:



**Correct - Create new graphs, ignore default graph:**
```python
import tensorflow as tf

g1 = tf.Graph()
g2 = tf.Graph()

with g1.as_default():
   # Define and Do something with g1 operations, tensors.....
   
with g2.as_default():
   # Define and Do something with g2 operations, tensors.....
```


**Correct - Get handle to default graph:**
```python
import tensorflow as tf

g1 = tf.get_default_graph()
g2 = tf.Graph()

with g1.as_default():
   # Define and Do something with g1 operations, tensors.....
   
with g2.as_default():
   # Define and Do something with g2 operations, tensors.....
```

   
   


## 2. Adding Inputs with Placeholder nodes

Placeholders act as if they are Tensor objects, but they do not have their values specified when created. Instead, they hold the place for a Tensor that will be fed at runtime, in effect becoming an "input" node.

In [5]:
import numpy as np

In [6]:
# Creates a placeholder vector of length 2 with data type int32
a = tf.placeholder(tf.int32, shape=[2], name="my_input")

# Use the placeholder as if it were any other Tensor object
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")

# Finish off the graph
d = tf.add(b, c, name="add_d")

In order to actually give a value to the placeholder, we'll use the *feed_dict* parameter in *Session.run()*. 

**The Feed dictionary:**
The parameter *feed_dict* is used to override *Tensor* values in the graph, and it excepts a Python dictionary object as input. 
- The keys in the dict are handles to Tensor objects that should be overridden
- The values can be numbers, strings, lists, or NumPy arrays.

In [8]:
# Open a TensorFlow Session
with tf.Session() as sess:
    
    # Create a dictionary to pass into 'feed_dict'
    # Key: 'a', the handle to the placeholder's output Tensor
    # Value: A vector with value [5, 3] and int32 data type
    
    input_dict = {a: np.array([5, 3], dtype=np.int32)}
    
    # Fetch the value of 'd' feeding the values of 'input_vector' into 'a'
    sess.run(d, feed_dict=input_dict)